# Python Intro - Final Exercise


In [15]:
import pyodbc
import pandas as pd
import numpy as np

In [36]:
### In windows, Using a ODBC DNS (predefined connection name)
conn = pyodbc.connect("DSN=College;"
                      "Server=DESKTOP-E6QQ4LT\SQLEXPRESS;"
                      "Database=College;"
                      "Trusted_Connection=yes;")


In [63]:
### Get the students table
students = pd.read_sql("SELECT * FROM Students;",conn) 
students.head(5)

,Student_id,First_name,Last_name,Gender
0,162,Elmira,Holston,F
1,154,Sierra,Van Vorst,F
2,28,Mycah,Bingham,F
3,54,Alia,Jones,F
4,31,Chanda,Fillmore,F


In [38]:
departments = pd.read_sql("SELECT * FROM Departments;",conn)
departments.head()

,Department_id,Department_name
0,1,English
1,2,Science
2,3,Arts
3,4,Sport


In [48]:
courses = pd.read_sql("SELECT * FROM Courses;",conn)
courses.head(5)

,Course_id,Course_name,Department_id,Teacher_id
0,1,English Begginers,1,15
1,2,Advanced English,1,15
2,3,Proffesional English,1,15
3,4,Mathematics 101,2,2
4,5,Matematics Advanced,2,13


In [47]:
classroom = pd.read_sql("SELECT * FROM Classroom;",conn)
classroom.head(5)

,Course_id,Student_id,Degree
0,1,202,92.031070
1,1,12,81.175916
2,1,232,82.396587
3,1,123,62.420449
4,1,274,72.569900


In [46]:
#inner join classroom with courses
classroom_courses = pd.merge(classroom, 
                             courses[['Course_id', 'Course_name', 'Department_id']], 
                             on='Course_id')
classroom_courses.head(5)

,Course_id,Student_id,Degree,Course_name,Department_id
0,1,202,92.031070,English Begginers,1
1,1,12,81.175916,English Begginers,1
2,1,232,82.396587,English Begginers,1
3,1,123,62.420449,English Begginers,1
4,1,274,72.569900,English Begginers,1


In [45]:
#inner join classroom_courses with departments 
classroom_courses_departments = pd.merge(classroom_courses,
                                        departments[['Department_id', 'Department_name']],
                                        on='Department_id')
classroom_courses_departments.head()

,Course_id,Student_id,Degree,Course_name,Department_id,Department_name
0,1,202,92.031070,English Begginers,1,English
1,1,12,81.175916,English Begginers,1,English
2,1,232,82.396587,English Begginers,1,English
3,1,123,62.420449,English Begginers,1,English
4,1,274,72.569900,English Begginers,1,English


#. Count the number of students on each department

In [64]:
#counting the number of students in each department
classroom_courses_departments.groupby('Department_id').agg({'Student_id': pd.Series.nunique})

,Student_id
Department_id,
1,84
2,200
3,137
4,81


## Q2. How many students have each course of the English department and the total number of students in the department?

## Q3. How many small (<22 students) and large (22+ students) classrooms are needed for the Science department?

## Q4. A feminist student claims that there are more male than female in the College. Justify if the argument is correct

## Q5. For which courses the percentage of male/female students is over 70%?

## Q6. For each department, how many students passed with a grades over 80?

## Q7. For each department, how many students passed with a grades under 60?

## Q8. Rate the teachers by their average student's grades (in descending order).

## Q9. Create a dataframe showing the courses, departments they are associated with, the teacher in each course, and the number of students enrolled in the course (for each course, department and teacher show the names).

## Q10. Create a dataframe showing the students, the number of courses they take, the average of the grades per class, and their overall average (for each student show the student name).